# 6C - Latent Gaussian Mixture Model with a 2D Auto-Encoder (AE-GMM)

In the previous notebooks #6A and #6B, we introduced respectively latent-reprensentation learning with auto-encoders (AE) and generative modeling in data space using GMM.
 
In the perspective of generative modeling, we have seen that auto-encoders can be used to learn a latent representation of the data, which can then be used to generate new samples by sampling from the latent space **appropriately** and decoding the samples back to the data space.
The major limitation of this approach is that the latent space is not guaranteed to be structured in a way that allows for meaningful sampling.

In this notebook, we will explore the concept of latent Gaussian Mixture Models (GMM) using an auto-encoder in two stages :
1. first, we train a simple auto-encoder based on MLP using PyTorch in the data space (2D) : **we will reuse the code from notebook #6A**
1. second, we train a genetaive model based on the learned latent representation, again using GMM (latent GMM) : **we will reuse the code from notebook #6B or make use of sklearn's GMM implementation.**

Finally, we will conclude on the need to train a more sophisticated generative model : a latent representation and a generative model trained all at once, e.g. using Variational Auto-Encoders (VAE, see Notebooks #7) or Diffusion Models (DDPM ... see Notebooks #8).

---
julien rabin @ greyc.ensicaen.fr 2025

In [1]:
import torch 
from torch import nn, Tensor

import numpy as np

import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True" # Allow multiple OpenMP threads (avoid conflicts between Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') used by PyTorch and scikit-learn)
from sklearn.datasets import make_moons
from sklearn.mixture import GaussianMixture

Data loader

In [3]:
def sample_data(n: int, model = 'radial_gmm') -> Tensor:
    data_dim = 2  # Dimension of the data
    if model == 'moons':
        from sklearn.datasets import make_moons
        return Tensor(make_moons(n_samples=n, noise=0.05)[0])
    elif model == 'circles':
        from sklearn.datasets import make_circles
        return Tensor(make_circles(n_samples=n, noise=0.05, factor=0.5)[0])
    elif model == '2gmm':
        n_samples = n
        n = n//3
        X1 = torch.randn(n, data_dim) @ torch.tensor([[.05, -0.02],[-0.02, .4]]) + torch.tensor([[.0, 1.0]]).view(1, data_dim)
        X2 = torch.randn(n_samples - n, data_dim) @ torch.tensor([[.3, 0.05],[0.05, .05]]) + torch.tensor([[-1.0, 0.]]).view(1, data_dim)
        return torch.cat((X1,X2), dim=0)  # Concatenate the two sets of samples
    elif model == 'radial_gmm':
        K = 8
        n_samples = n
        samples_per_component = n_samples // K
        remainder = n_samples % K
        all_samples = []

        for k in range(K):
            radius = 3.
            # Angle for the mean on a circle
            theta = 2 * np.pi * k / K
            cs = np.cos(theta)
            sn = np.sin(theta)

            # Radial direction unit vector
            radial = torch.tensor([cs, sn], dtype=float)#.view(data_dim, 1)
            tangential = torch.tensor([-sn, cs], dtype=float)#.view(data_dim, 1)

            mean = radius * radial
            
            # Covariance matrix: elongated along radial direction
            cov = 0.3 * torch.outer(radial, radial).to(float) + 0.05 * torch.outer(tangential, tangential).to(float)

            # Generate samples
            n = samples_per_component + (remainder if k == K else 0)
            samples = torch.randn(n, data_dim, dtype=float) @ torch.linalg.cholesky(cov).to(float) + mean.to(float)
            all_samples.append(samples)

        return torch.cat(all_samples, dim=0).to(torch.float32)
    else:
        raise ValueError(f"Unknown model: {model}.")


Useful functions for visualization

In [4]:
def plot_data_comparison(x_data : np.ndarray, x_model : np.ndarray, colors = None):
    if colors is None:
        colors = 'C0' #np.random.rand(x_data.shape[0])
    fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharex=True, sharey=True)

    # Original data
    axes[0].scatter(x_data[:, 0], x_data[:, 1], s=10, c=colors)
    axes[0].set_title('Data Samples')
    axes[0].set_xlim(-3.0, 3.0)
    axes[0].set_ylim(-3.0, 3.0)

    # VAE reconstruction
    axes[1].scatter(x_model[:, 0], x_model[:, 1], s=10, c=colors)
    axes[1].set_title('Model Output Samples')
    axes[1].set_xlim(-3.0, 3.0)
    axes[1].set_ylim(-3.0, 3.0)

    # Global title
    fig.suptitle("Original vs Model samples", fontsize=24)
    plt.tight_layout()
    plt.subplots_adjust(top=0.88)

    return fig, axes


In [5]:
def define_colors(x_data: np.ndarray ) -> np.ndarray:
    cmap = plt.get_cmap("rainbow")
    values = x_data[:, 0]  # Use the first dimension for coloring
    normalized = (values - values.min()) / (values.max() - values.min())
    colors = cmap(normalized)
    return colors[:, :3]

# Part II : Latent Gaussian Mixture Model using Auto-Encoder


Neural Networks Architecture (same as in notebook #0)

In [6]:
class AE(nn.Module):
    def __init__(self, dim: int = 2, h: int = 64, latent_dim: int = 1):
        super().__init__()
        
        self.lat_dim = latent_dim
        
        self.encoder = nn.Sequential(
            nn.Linear(dim, latent_dim)
        ) # Output is the latent representation
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, h)
        )
    
    def forward(self, x: Tensor) -> Tensor: # AE mode : encode then decode, using the reparameterization trick
        # Note : x is of shape (batch_size, dim)
        
        z = self.encode(x) # latent shape (batch_size, latent_dim)
        
        # Decode the latent space representation to data space
        y = self.decode(z) # Output is of shape (batch_size, dim)
        
        return y
    
    def encode(self, x: Tensor) -> tuple[Tensor, Tensor]:
        # Encode input x to latent space
        z = self.encoder(x)
        return z
    
    def decode(self, z: Tensor) -> Tensor:
        # Decode latent space representation z to data space
        y = self.decoder(z)
        return y
    
    def sample_gauss(self, n: int, mu : Tensor = None, std : Tensor = None) -> Tensor:
        # Sample from the latent space distribution N(mu, std I_d)
        # Note : there is no reason why the learned latent distribution should be gaussian as AE are not generative model and this is not enforced during training !
        if mu is None :
            mu = torch.zeros((n, self.lat_dim))
        if std is None :
            std = torch.ones((n, self.lat_dim))
        assert mu.shape == (n, self.lat_dim) and std.shape == (n, self.lat_dim), "mu and std must have shape (n, latent_dim)"
        # Sample from the latent space distribution
        z = ...
        return self.decoder(z)
    
    def sample_uniform(self, n: int, low: float = -1.0, high: float = 1.0) -> Tensor:
        assert low < high, "low must be less than high"
        # Sample uniformly from the latent space
        z = ...
        return self.decoder(z)
    

Number of parameters

In [7]:
model = AE(dim=2, h=64, latent_dim=1)
n_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {n_params}")

Number of trainable parameters: 131


## Stage 1 : Latent Representation learning with Auto encoder Training

### Auto-Encoder model initialization

In [ ]:
torch.manual_seed(0) # For reproducibility

dataset_name = 'moons' # moons, circles, 2gmm, radial_gmm

latent_dim = 1
model = ...

learning_rate = ...
optimizer = ... # gradient descent optimizer
criterion = ... # reconstruction loss

### Auto-Encoder model training

In [ ]:
n_iter = 1_000
batch_size = 256

rec_losses = []

pbar = tqdm(range(n_iter), desc="Training AE")
for _ in pbar :
    x = ...
    y = ...
    
    optimizer.zero_grad()
    rec_loss = criterion(x, y) # Reconstruction loss
    rec_loss.backward()
    optimizer.step()
    
    rec_losses.append(rec_loss.item())
    
    pbar.set_postfix({
        'loss'  : f"{rec_losses[-1]:.4f}",
    })

Training AE:   0%|          | 0/1000 [00:00<?, ?it/s]/Users/rabin/doc/TeX/Cours_ENSICAEN/Cours_deep_avancé/prepa_TP/demo_generative_model_2D/venv_demo_deep_gen_model/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256, 64])) that is different to the input size (torch.Size([256, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Training AE:   0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (2) must match the size of tensor b (64) at non-singleton dimension 1

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 5))
ax.semilogy(rec_losses, label='Reconstruction Loss', color='blue')
ax.set_title('Losses during AE Training')
ax.set_xlabel('Iteration')
ax.set_ylabel('Loss Value')
ax.legend()
plt.show()

### Evaluation of the Auto-Encoder by reconstruction of data samples

In [ ]:
# Generate samples from the data model
n = 1_000
x = ...

# reconstruction of the training data : using only the mean of the learned distribution
y = ...
colors = define_colors(x.detach())
fig, axes = plot_data_comparison(x,y, colors=colors)
fig.suptitle("AE reconstruction of data samples", fontsize=24)



### visualization of the latent space distribution of the data

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10), sharex=True, sharey=True)

n = 1_000
# Generate samples from the training data
x = ...

# reconstruction of the training data
z = ...
eps = torch.randn_like(z).detach().numpy() # Standard gaussian samples with same dimension for comparison
print(eps.shape, z.shape)

if model.lat_dim >= 2:
    axes[0].scatter(eps[:, 0], eps[:, 1], s=10, color='red')
    axes[0].set_title(f'standard gaussian latent space (first 2 dimensions)')
    axes[0].set_xlim(-3.0, 3.0)
    axes[0].set_ylim(-3.0, 3.0)

    z = z.detach().numpy()
    axes[1].scatter(z[:, 0], z[:, 1], s=10)
    axes[1].set_title(f'AE latent (first 2 dimensions)')
    axes[1].set_xlim(-3.0, 3.0)
    axes[1].set_ylim(-3.0, 3.0)
    
else: # 1D plot
    t = np.linspace(-3,3, 100)
    axes[0].hist(eps[:, 0],bins=t,color='r')
    axes[0].set_title(f'standard gaussian latent space (1D here)')

    z = z.detach().numpy()
    axes[1].hist(z[:, 0],bins=t)
    axes[1].set_title(f'AE latent (1D here)')
    

## Stage 2 : GMM fitting in the latent space

### Generative model (GMM) initialization and training using scikit-learn (see previous notebook for a detailed explanation and implementation)

In [ ]:
K_GMM = ...
n = ... # number of data points to sample
x_data = ... # sample data points to create a dataset for GMM fitting
z_data = ... # compute the latent representation of the data points using the encoder of the trained AE

# same as notebook 6B but with sklearn and using latent representation from AE
gmm = GaussianMixture(n_components=K_GMM, covariance_type='full', max_iter=n_iter, random_state=42)
gmm.fit(z_data)  # Fit the GMM to the data
print(f"Sklearn GMM parameters:")
print(f"mu_k : {gmm.means_}\ncov_k : {gmm.covariances_}\npi_k : {gmm.weights_}")


### GMM sampling in the latent space and decoding to data space

In [ ]:
assert model.lat_dim == 1, "simple analysis in 1D"

n = 1_000
# Generate samples from the training data
x_data = sample_data(n, model=dataset_name).detach().numpy()

# generate gmm latent codes and then decode into data space
z, label = gmm.sample(n_samples=n)
x_model = model.decode(Tensor(z)).detach().numpy()

fig, axes = plot_data_comparison(x_data, x_model, colors=define_colors(label[:,np.newaxis]))
fig.legend()

fig2 = plt.figure(figsize=(5, 5))
for k in range(K_GMM):
    plt.hist(z[label==k], label=f'GMM #{k}')
fig2.suptitle("GMM latent space samples", fontsize=24)
fig2.legend()


#### Exercise
Discuss the advantages and limitations of such latent GMM considering the following questions :
- about training ?
- about the quality of the generative model ?
- about the selection of the number of components to choose ?
- what is the metric used to evaluate the quality of the generative model ?
- what is the metric used to fit the GMM in the latent space ?
- what are the latent variables modeled by the GMM ? what are the latent variables modeled by the AE ? Are they the same ?